In [5]:
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from typing import Annotated
from typing_extensions import TypedDict
# 検索ツールの定義
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver

# ===== Stateクラスの定義 =====
class State(TypedDict):
    messages: Annotated[list, add_messages]

# ===== グラフの構築 =====
def build_graph(model_name):
    # ソースコードを記述
    tool = TavilySearchResults(max_results=2)
    tools = [tool]

    graph_builder = StateGraph(State)
    # 言語モデルの定義
    llm = ChatOpenAI(model_name=MODEL_NAME)

    # 変更点：ツール定義の紐づけ
    llm_with_tools = llm.bind_tools(tools)

    # チャットボットノードの作成
    def chatbot(state: State):
        return {"messages": [llm_with_tools.invoke(state["messages"])]}

    # グラフにチャットボットノードを追加
    graph_builder.add_node("chatbot", chatbot)
    # return graph_builder
    # ツールノードの作成
    tool_node = ToolNode(tools)

    # グラフにツールノードを追加
    graph_builder.add_node("tools", tool_node)

    # 条件付エッジの作成
    graph_builder.add_conditional_edges(
        "chatbot",
        tools_condition, # ツール呼出と判断したらツールノードを呼ぶ
    )

    # ツールが呼び出されるたびに、チャットボットに戻って次のステップを決定
    # ツールからチャットボットへの戻りエッジを作成
    graph_builder.add_edge("tools", "chatbot")

    # 開始ノードの指定
    graph_builder.set_entry_point("chatbot")

    # 記憶を持つ実行可能なステートグラフの作成
    memory = MemorySaver()
    graph = graph_builder.compile(checkpointer=memory)
    return graph

# ===== グラフ実行関数 =====
def stream_graph_updates(graph: StateGraph, user_input: str):
    # ソースコードを記述
    events = graph.stream(
        {"messages": [("user", user_input)]},
        {"configurable": {"thread_id": "1"}},
        stream_mode="values")
    # 結果をストリーミングで得る
    for event in events:
        print(event["messages"][-1].content, flush=True)

# ===== メイン実行ロジック =====
# 環境変数の読み込み
load_dotenv("../.env")
os.environ['OPENAI_API_KEY'] = os.environ['API_KEY']

# モデル名
MODEL_NAME = "gpt-4o-mini" 

# グラフの作成
# ソースコードを記述
graph = build_graph(MODEL_NAME)

# メインループ
# ソースコードを記述
# チャットボットのループ
while True:
    user_input = input("質問:")
    if user_input.strip()=="":
        print("ありがとうございました!")
        break
    stream_graph_updates(graph, user_input)


こんにちは
こんにちは！今日はどのようなことをお手伝いできますか？
1たす999は？
1たす999は1000です。
東京観光について検索結果を教えて

[{"title": "【保存版】東京観光おすすめ30選！絶対外せない定番スポットを ...", "url": "https://www.e-myholiday.com/kanto/tokyo/spots/", "content": "【お台場・豊洲】周辺エリア\n 迫力溢れるマグロのせりを見学！豊洲市場\n アート作品への没入体験！チームラボプラネッツ TOKYO DMM\n 最新アトラクションが勢ぞろい！東京ジョイポリス\n レインボーブリッジを望む絶景！お台場海浜公園&台場公園\n 【三鷹・多摩】周辺エリア\n ミシュランガイド三ツ星の山！高尾山\n ありのままの動物を観察！多摩動物公園\n 桜の名所としても有名！井の頭恩賜公園\n 【シチュエーション別】東京のおすすめ観光スポット\n 「雨の日でも楽しみたい！」屋内観光スポット\n 「一人でじっくり」東京で芸術と文化を味わうスポット\n 「女子なら一度は行ってみたい！」東京のおしゃれスポット\n 「カップルで最高の思い出を」東京の夜景スポット\n 東京観光に関するよくある質問\n Q.東京を満喫するにはどのぐらいの日数が必要？\n Q.東京観光の際に注意すべき点は？\n Q.どこに行くにも予約は必須？\n 東京の魅力は無限大！行くたびに新しい発見を\n 東京行きのホテル&航空券を検索！\n\n## 東京観光おすすめ30選！\n\n## 【東京駅・皇居・銀座】周辺エリア\n\n### 1東京の玄関口！東京駅\n\n東京駅は、1914年に竣工した壮麗な赤レンガ駅舎がシンボルの東京の玄関口です。2012年に創建当時の姿に復元され、その美しさがさらに際立ちました。夜にはライトアップされ、昼のとは異なる幻想的な雰囲気を醸し出します。駅舎内には、東京駅の歴史を紹介する「東京ステーションギャラリー」に立ち寄れば、東京駅の魅力が一層増すことでしょう。また、駅舎の上層階にあるホテルに宿泊すれば、東京の街並みを一望しながら特別な時間を過ごせますよ。単なる駅としてではなく、東京の歴史や文化を感じるスポットとして観光してみてはいかがでしょうか。 [...] ### 4四季折々の美しさ